In [ ]:
import mlrun

mlrun.set_env_from_file("mlrun.env")

project = mlrun.get_or_create_project(name="sense-private-git", context="./")

### Create Kubernetes Secret

You only need to add this once per project. Do not commit this to git

In [ ]:
secrets = {'GIT_TOKEN': 'XXXXXXXXXXXXXXXXXXXXX'}
project.set_secrets(secrets=secrets, provider="kubernetes")

### Code Overview

In [ ]:
!cat module.py

In [ ]:
!cat test.py

### Create Function with Git Repo and Secret

This is creating a new function to run the job

In [ ]:
fn = mlrun.new_function(
    name="git-test",
    kind="job",
    image="mlrun/mlrun",
)

This is adding your git repo to the job and specifying to pull it at runtime. The handler is in the format of `<FILE>.<FUNCTION>`. In this case, my `test.py` file has a function called `handler`

In [ ]:
fn.with_source_archive(
    source="git://github.com/igz-us-sales/sense-git.git",
    handler="test.handler",
    pull_at_runtime=True
)

This is running the job and adding your secret. The secret will be pulled at runtime from the Kubernetes cluster

In [ ]:
fn.run(
    local=False,
    runspec=mlrun.new_task().with_secrets('kubernetes', ['GIT_TOKEN'])
)